# Merging and Cleaning Citi Bike Data
## Lauren Brodsky 
## 4/2/25

In [4]:
# Packages used:
import pandas as pd
import warnings
warnings.filterwarnings("ignore") # hides datetime format warning (handled by 'coerce')

# Data Cleaning for Citit Bike Data

In [7]:
# load the data 
citibike_data = pd.read_csv('NYC_citibike_2023.csv',low_memory=False) #< for the warning since it's a large file

In [8]:
# check for duplicated rows 
duplicate_rows = citibike_data.duplicated()
print(duplicate_rows.sum())

0


In [ ]:
#convert started_at to datetime
citibike_data['started_at'] = pd.to_datetime(citibike_data['started_at'], errors='coerce')

In [ ]:
#convert ended_at to datetime 
citibike_data['ended_at'] = pd.to_datetime(citibike_data['ended_at'], errors='coerce')

# Aggergate Data 

In [9]:
# get the date from start time
citibike_data['date'] = citibike_data['started_at'].dt.date

# aggregate data using groupby method 
# NamedAgg > this defines the column names and the aggregation method 
# aggfunc=lambda x > this applies the function to the data in each group
# x > represents the data in the column for each group

daily_data = citibike_data.groupby('date').agg(
    total_rides=pd.NamedAgg(column='ride_id', aggfunc='size'), #< aggfunc= size counts the number per group   # Count total rides
    classic_bikes=pd.NamedAgg(column='rideable_type', aggfunc=lambda x: (x == 'classic_bike').sum()),  # counts classic bikes
    electric_bikes=pd.NamedAgg(column='rideable_type', aggfunc=lambda x: (x == 'electric_bike').sum()),  # counts electric bikes
    docked_bikes=pd.NamedAgg(column='rideable_type', aggfunc=lambda x: (x == 'docked_bike').sum()),  # counts docked bikes
    average_ride_duration =pd.NamedAgg(column='ride_id', aggfunc=lambda x: (citibike_data.loc[x.index, 'ended_at'] - citibike_data.loc[x.index, 'started_at']).mean().total_seconds() / 60),  # finds the avg ride time in min using ended_at and started_at columns
    rides_by_members=pd.NamedAgg(column='member_casual', aggfunc=lambda x: (x == 'member').sum()),  # counts rides by members
    rides_by_casual=pd.NamedAgg(column='member_casual', aggfunc=lambda x: (x == 'casual').sum()) ) # counts rides by casual users

In [10]:
# check the data > notice there are 365 rows /days of data 
daily_data

,total_rides,classic_bikes,electric_bikes,docked_bikes,average_ride_duration,rides_by_members,rides_by_casual
date,,,,,,,
2023-01-01,1444,1129,288,27,19.840028,991,453
2023-01-02,1714,1410,294,10,14.822637,1274,440
2023-01-03,1444,1178,263,3,13.310942,1294,150
2023-01-04,2109,1732,366,11,13.600284,1808,301
2023-01-05,2087,1707,366,14,11.183517,1827,260
...,...,...,...,...,...,...,...
2023-12-27,1142,1106,36,0,12.485989,985,157
2023-12-28,1384,1350,34,0,12.593208,1171,213
2023-12-29,2068,2017,51,0,14.490329,1603,465


# NEW citi bike data cleaning 

In [12]:
# check data dtypes
print(daily_data.dtypes)

total_rides                int64
classic_bikes              int64
electric_bikes             int64
docked_bikes               int64
average_ride_duration    float64
rides_by_members           int64
rides_by_casual            int64
dtype: object


In [13]:
# check for missing values > none
daily_data.isnull().sum()	

total_rides              0
classic_bikes            0
electric_bikes           0
docked_bikes             0
average_ride_duration    0
rides_by_members         0
rides_by_casual          0
dtype: int64

In [14]:
# check for duplicates > none
daily_data.index.duplicated().sum()	

0

In [15]:
# reset index > for merging 
daily_data = daily_data.reset_index()
daily_data

,date,total_rides,classic_bikes,electric_bikes,docked_bikes,average_ride_duration,rides_by_members,rides_by_casual
0,2023-01-01,1444,1129,288,27,19.840028,991,453
1,2023-01-02,1714,1410,294,10,14.822637,1274,440
2,2023-01-03,1444,1178,263,3,13.310942,1294,150
3,2023-01-04,2109,1732,366,11,13.600284,1808,301
4,2023-01-05,2087,1707,366,14,11.183517,1827,260
...,...,...,...,...,...,...,...,...
360,2023-12-27,1142,1106,36,0,12.485989,985,157
361,2023-12-28,1384,1350,34,0,12.593208,1171,213
362,2023-12-29,2068,2017,51,0,14.490329,1603,465
363,2023-12-30,1646,1600,46,0,12.621507,1252,394


In [16]:
# save file to csv 
daily_data.to_csv('cleaned_citibike_daily.csv', index=False)

# Merge Citi Bike Data with Weather Data 

In [18]:
# load in weather data 
weather_data = pd.read_csv('final_weather_data_scraped.csv')

In [19]:
# rename weather data date to match > so we can merge on date
weather_data.rename(columns={'Date': 'date'}, inplace=True)

#rename weather columns to match
weather_data.columns = ['avg_temperature', 'precipitation_in', 'wind_speed_mph', 'date']
weather_data #check

,avg_temperature,precipitation_in,wind_speed_mph,date
0,51.77,0.33,18,2023-01-01
1,52.32,0.00,10,2023-01-02
2,49.17,0.02,9,2023-01-03
3,53.79,0.41,14,2023-01-04
4,45.52,0.03,13,2023-01-05
...,...,...,...,...
360,44.60,0.00,20,2023-12-27
361,49.71,1.31,29,2023-12-28
362,48.70,0.09,17,2023-12-29
363,43.50,0.00,20,2023-12-30


In [20]:
# make sure both dates are set to datetime 
weather_data['date'] = pd.to_datetime(weather_data['date'])
daily_data['date'] = pd.to_datetime(daily_data['date'])

In [21]:
# merge the data 
merged_data = pd.merge(daily_data, weather_data, on='date', how='inner')
merged_data #final check

,date,total_rides,classic_bikes,electric_bikes,docked_bikes,average_ride_duration,rides_by_members,rides_by_casual,avg_temperature,precipitation_in,wind_speed_mph
0,2023-01-01,1444,1129,288,27,19.840028,991,453,51.77,0.33,18
1,2023-01-02,1714,1410,294,10,14.822637,1274,440,52.32,0.00,10
2,2023-01-03,1444,1178,263,3,13.310942,1294,150,49.17,0.02,9
3,2023-01-04,2109,1732,366,11,13.600284,1808,301,53.79,0.41,14
4,2023-01-05,2087,1707,366,14,11.183517,1827,260,45.52,0.03,13
...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,1142,1106,36,0,12.485989,985,157,44.60,0.00,20
361,2023-12-28,1384,1350,34,0,12.593208,1171,213,49.71,1.31,29
362,2023-12-29,2068,2017,51,0,14.490329,1603,465,48.70,0.09,17
363,2023-12-30,1646,1600,46,0,12.621507,1252,394,43.50,0.00,20


In [22]:
# save the final data 
merged_data.to_csv('final_merged_data.csv', index=False)